In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import itertools
import sys
sys.path.append('..')
from security_game.green_security_game import GreenSecurityGame
from security_game.infra_security_game import InfraSecurityGame

# GSG Testing

In [10]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
lat_min, lon_min = 2.05522, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]

schedule_form_kwargs = {
    "schedule_form": True,
    "attacker_animal_value":  2150, 
    "defender_animal_value": 10000, 
    "defender_step_cost": 15.7, 
    "simple": False,
    "attacker_penalty_factor": 4,
    "defender_penalty_factor": 4,
    "extra_coverage_weight":0.9
}

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=5, num_rows=5, num_columns=5)
gsg.generate(num_attackers=2, num_defenders=2, home_base_assignments=[(3,3),(1,1)], num_timesteps=6, generate_utility_matrix=True, general_sum=True, **schedule_form_kwargs)
#self, num_attackers, num_defenders, home_base_assignments, num_timesteps, interdiction_protocol=None, defense_time_threshold=2, generate_utility_matrix=False, schedule_form=False, general_sum=False, attacker_animal_value=1, defender_animal_value=1, defender_step_cost=0
# generate True, schedule False, general false works
# generate True, Schedule false, General True works
# generate False, schedule False, general false works
# genereate False, schedule false, general true works
# generate false, schedule true, general false
# generate true, schedule true




15.7


In [6]:
gsg.graph.nodes[0]

{'score': 1.8051539912005028, 'position': (4, 2)}

In [9]:
print(gsg.attacker_strategies)
print(gsg.defender_strategies)
print(gsg.attacker_utility_matrix)
print(gsg.defender_utility_matrix)
print(gsg.utility_matrix)
print(gsg.schedule_form_dict)

None
None
None
None
None
{'schedules': {0: [({0}, 0), ({4}, 0)], 1: [({1}, 0), ({1, 3}, 0), ({1, 4}, 0), ({3}, 0), ({3, 4}, 0), ({4}, 0)]}, 'target_utilities': array([[-1.        , -0.27158774, -0.20612813, -0.29387187, -0.44428969],
       [-0.25      , -0.06789694, -0.05153203, -0.07346797, -0.11107242],
       [ 0.25      ,  0.06789694,  0.05153203,  0.07346797,  0.11107242],
       [ 1.        ,  0.27158774,  0.20612813,  0.29387187,  0.44428969]]), 'attacker_utility_matrix': array([[0.25      , 0.06789694, 0.20612813, 0.29387187, 0.44428969],
       [0.25      , 0.06789694, 0.20612813, 0.07346797, 0.44428969],
       [0.25      , 0.06789694, 0.20612813, 0.29387187, 0.11107242],
       [0.25      , 0.27158774, 0.20612813, 0.07346797, 0.44428969],
       [0.25      , 0.27158774, 0.20612813, 0.07346797, 0.11107242],
       [0.25      , 0.27158774, 0.20612813, 0.29387187, 0.11107242],
       [1.        , 0.06789694, 0.20612813, 0.29387187, 0.11107242],
       [1.        , 0.06789694, 

In [9]:
gsg.schedule_form_dict["target_utilities"]

array([[-10000.        ,  -2715.87743733,  -2061.28133705,
         -2938.71866295,  -4442.89693593],
       [ -2500.        ,   -678.96935933,   -515.32033426,
          -734.67966574,  -1110.72423398],
       [   537.5       ,    145.97841226,    110.79387187,
           157.95612813,    238.80571031],
       [  2150.        ,    583.91364903,    443.17548747,
           631.82451253,    955.22284123]])

In [3]:
def get_full_path_with_dwell_and_return(graph, start_node, targets, dwell_time):
    """Returns a full path that visits all targets (with dwell time) and returns to base."""
    min_total_path = None
    min_total_cost = float("inf")

    for perm in itertools.permutations(targets):
        path = [start_node]
        current = start_node
        total_cost = 0
        dwell_steps = []

        try:
            # Move through targets in order
            for node in perm:
                segment = nx.shortest_path(graph, source=current, target=node)
                if segment[0] == current:
                    segment = segment[1:]
                path.extend(segment)
                total_cost += len(segment)
                # Add dwell time (minus 1 because standing on the node already counts as 1)
                path.extend([node] * (dwell_time - 1))
                total_cost += (dwell_time - 1)
                current = node

            # Return to home base
            return_segment = nx.shortest_path(graph, source=current, target=start_node)
            if return_segment[0] == current:
                return_segment = return_segment[1:]
            path.extend(return_segment)
            total_cost += len(return_segment)

            # Update if this path is the best so far
            if total_cost < min_total_cost:
                min_total_cost = total_cost
                min_total_path = list(path)

        except nx.NetworkXNoPath:
            continue  # Skip infeasible permutations

    return min_total_path, min_total_cost
    

def find_valid_schedules(num_timesteps, graph, target_nodes, start_node, simple=False):
    def backtrack(current_set, remaining_nodes):
            if current_set:
                path, total_cost = get_full_path_with_dwell_and_return(graph, start_node, current_set, 2)

                if path is not None and total_cost <= num_timesteps:
                    valid_schedules.append(set(current_set))

            for i, node in enumerate(remaining_nodes):
                backtrack(current_set | {node}, remaining_nodes[i+1:])

    if simple:
        defendable_targets = set()
        for target in target_nodes:
            # Compute shortest round-trip path length
            path_to_target = nx.shortest_path_length(graph, source=start_node, target=target)
            round_trip_time = 2 * path_to_target  # Going to target + returning
            
            # Check if the defender can reach, defend, and return within time limit
            if round_trip_time + 2 <= num_timesteps:
                defendable_targets.add(target)
        return [(t,) for t in defendable_targets]
        
    else:
        valid_schedules = []

        backtrack(set(), target_nodes)
        return valid_schedules


In [6]:
find_valid_schedules(8, gsg.graph, [t.node for t in gsg.targets], 18, simple=True)

[(1,), (2,), (3,), (4,)]

# ISG Testing

In [1]:
import geopandas as gpd
import sys
import pandas as pd
import requests
import osmnx as ox
import networkx as nx
import random
sys.path.append('..')
from security_game.infra_security_game import InfraSecurityGame
from security_game.target import Target
# pd.options.display.max_rows = 4000

In [2]:
gdf = gpd.read_file("nyc_power.geojson")
df = gdf[["id","name","operator","power","voltage","substation","transformer","generator:method","circuits","wires","area","man_made","height","location","phases","line_management","line_attachment","devices","emergency","foot","geometry"]]
df = df[~df["generator:method"].str.contains("photovoltaic", na=False)]
df_simple = df[["id","power","geometry"]]
df_nodes = df_simple[df_simple["id"].str.contains("node")].reset_index()
df_nodes["x"] = [df_nodes["geometry"][i].x for i in range(len(df_nodes))]
df_nodes["y"] = [df_nodes["geometry"][i].y for i in range(len(df_nodes))]
df_nodes = df_nodes.drop("geometry", axis=1)
# Define a projected CRS (e.g., UTM Zone 18N for NYC)
projected_crs = "EPSG:32618"  # UTM Zone 18N (suitable for New York)
geographic_crs = "EPSG:4326"  # Standard lat/lon

df_ways = df_simple[df_simple["id"].str.contains("way")].reset_index(drop=True)
df_ways = df_ways.set_geometry("geometry").to_crs(projected_crs)  # Reproject to projected CRS
df_ways["centroid"] = df_ways["geometry"].centroid  # Compute centroid in projected CRS
df_ways = df_ways.set_geometry("centroid").to_crs(geographic_crs)  # Convert back to lat/lon
df_ways["x"] = df_ways["centroid"].x
df_ways["y"] = df_ways["centroid"].y
df_ways = df_ways.drop(columns=["geometry", "centroid"])  # Drop unnecessary columns

df_combined = pd.concat([df_nodes, df_ways], ignore_index=True)

In [3]:
nj_blocks_gdf = gpd.read_file("tl_2020_34_tabblock20.shp")

In [4]:
def assign_population(power_df, block_gdf, mode="block", radius=None):
    """
    Assigns population to power features based on Census block data.

    Parameters:
    - power_df: DataFrame with columns ['id', 'x', 'y'] representing power feature locations.
    - block_gdf: GeoDataFrame with block geometries and population data.
    - mode: "block" assigns population of the block containing the feature; "radius" sums population of blocks within radius.
    - radius: Radius in meters for "radius" mode (ignored for "block" mode).

    Returns:
    - A new DataFrame with an added 'population' column.
    """

    # Convert power features into a GeoDataFrame
    power_gdf = gpd.GeoDataFrame(
        power_df,
        geometry=gpd.points_from_xy(power_df.x, power_df.y),
        crs=block_gdf.crs  # Ensure matching coordinate reference system
    )

    if mode == "block":
        # Spatial join to find the containing block
        joined = gpd.sjoin(power_gdf, block_gdf[['POP20', 'geometry']], how="left", predicate="within")
        power_df['population'] = joined['POP20']

    elif mode == "radius":
        if radius is None:
            raise ValueError("Radius must be specified for 'radius' mode.")

        # Convert meters to CRS units (assumes projected CRS, otherwise use geopy or pyproj for accurate conversion)
        power_gdf = power_gdf.to_crs(epsg=3857)  # Convert to meters-based CRS
        block_gdf = block_gdf.to_crs(epsg=3857)

        # Create buffer zones and find intersecting blocks
        power_gdf['buffer'] = power_gdf.geometry.buffer(radius)
        power_df['population'] = power_gdf['buffer'].apply(
            lambda buf: block_gdf[block_gdf.geometry.intersects(buf)]['POP20'].sum()
            if not block_gdf[block_gdf.geometry.intersects(buf)].empty else float('nan')
        )

    else:
        raise ValueError("Invalid mode. Choose 'block' or 'radius'.")

    return power_df[['id', 'x', 'y', 'power', 'population']]

power_df = assign_population(df_combined, nj_blocks_gdf, mode="block", radius=None)

# Define min/max bounds of power features
minx, miny, maxx, maxy = power_df["x"].min(), power_df["y"].min(), power_df["x"].max(), power_df["y"].max()

# Filter blocks within this bounding box
nj_blocks = nj_blocks_gdf.cx[minx:maxx, miny:maxy]


In [7]:
# Power feature weights
POWER_WEIGHTS = {
    "plant": 2.0,
    "generator": 1.0,
    "line": 0.5,
    "cable": 0.25,
    "minor_line": 0.1,
    "tower": 0.75,
    "pole": 0.25,
    "substation": 1.0,
    "transformer": 0.5,
    "switchgear": 0.3,
    "busbar": 0.1,
    "bay": 0.1,
    "converter": 0.1,
    "insulator": 0.1,
    "portal": 0.1,
    "connection": 0.1,
    "compensator": 0.3,
    "rectifier": 0.1,
    "inverter": 0.1,
    "storage": 0.05,
}

schedule_form_kwargs = {
    "schedule_form": True,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
    "simple": False,
    "attacker_penalty_factor": 4,
    "defender_penalty_factor": 4,
    "extra_coverage_weight":0.9
}

# Bounding box for Hoboken, NJ
bbox_hoboken_small = (40.752635, 40.745600, -74.030386,-74.043903)

isg = InfraSecurityGame(power_df, nj_blocks, POWER_WEIGHTS, bbox=bbox_hoboken_small)
isg.generate(num_attackers=2, num_defenders=2, home_base_assignments=[8,30], num_timesteps=6, generate_utility_matrix=True, general_sum=True, **schedule_form_kwargs)
# generate True, schedule False, general false works
# generate True, Schedule false, General True works
# generate False, schedule False, general false works 
# genereate False, schedule false, general true works
# generate false, schedule true ?
# generate true, schedule true ?

C:\Users\Noah\GameGenerators\notebooks\..\security_game\infra_security_game.py:55: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  self.graph = ox.graph_from_bbox(north, south, east, west, network_type="drive")
C:\Users\Noah\GameGenerators\notebooks\..\security_game\infra_security_game.py:55: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  self.graph = ox.graph_from_bbox(north, south, east, west, network_type="drive")
C:\Users\Noah\GameGenerators\notebooks\..\security_game\infra_security_game.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [8]:
# print(isg.attacker_strategies[:5])
# print(isg.defender_strategies[:5])
# print(isg.attacker_utility_matrix)
# print(isg.defender_utility_matrix)
# print(isg.utility_matrix)
# print(isg.schedule_form_dict)

print(isg.attacker_strategies)
print(isg.defender_strategies)
print(isg.attacker_utility_matrix)
print(isg.defender_utility_matrix)
print(isg.utility_matrix)
print(isg.schedule_form_dict)

None
None
None
None
None
{'schedules': {0: [({9}, 65.0), ({9, 12}, 130.0), ({9, 14}, 130.0), ({9, 41}, 130.0), ({12}, 130.0), ({14}, 130.0), ({41}, 130.0)], 1: [({16}, 130.0), ({16, 15}, 130.0), ({25}, 130.0), ({25, 15}, 130.0), ({12}, 130.0), ({12, 15}, 130.0), ({15}, 65.0)]}, 'target_utilities': array([[ -5520.    , -15076.5   ,  -9004.5   ,  -5313.    ,  -3674.25  ,
        -12972.    ,     -0.    ,  -3053.25  ,  -2794.5   ,  -5313.    ,
        -10626.    ],
       [ -1380.    ,  -3769.125 ,  -2251.125 ,  -1328.25  ,   -918.5625,
         -3243.    ,     -0.    ,   -763.3125,   -698.625 ,  -1328.25  ,
         -2656.5   ],
       [   840.    ,   2294.25  ,   1370.25  ,    808.5   ,    559.125 ,
          1974.    ,      0.    ,    464.625 ,    425.25  ,    808.5   ,
          1617.    ],
       [  3360.    ,   9177.    ,   5481.    ,   3234.    ,   2236.5   ,
          7896.    ,      0.    ,   1858.5   ,   1701.    ,   3234.    ,
          6468.    ]]), 'attacker_utility_matrix': 

In [8]:
isg.defender_strategies

array([[[ 8, 30],
        [ 8, 30],
        [ 8, 30],
        [ 8, 30],
        [ 8, 30],
        [ 8, 30]],

       [[ 8, 30],
        [ 8, 30],
        [ 8, 11],
        [ 8, 12],
        [ 8, 15],
        [ 8, 30]],

       [[ 8, 30],
        [ 8, 11],
        [ 8, 11],
        [ 8, 12],
        [ 8, 15],
        [ 8, 30]],

       [[ 8, 30],
        [ 8, 11],
        [ 8, 12],
        [ 8, 12],
        [ 8, 15],
        [ 8, 30]],

       [[ 8, 30],
        [ 8, 11],
        [ 8, 12],
        [ 8, 15],
        [ 8, 15],
        [ 8, 30]],

       [[ 8, 30],
        [ 8, 11],
        [ 8, 12],
        [ 8, 15],
        [ 8, 30],
        [ 8, 30]],

       [[ 8, 30],
        [ 8, 30],
        [23, 30],
        [14, 30],
        [ 9, 30],
        [ 8, 30]],

       [[ 8, 30],
        [ 8, 30],
        [23, 11],
        [14, 12],
        [ 9, 15],
        [ 8, 30]],

       [[ 8, 30],
        [ 8, 11],
        [23, 11],
        [14, 12],
        [ 9, 15],
        [ 8, 30]],

       [[ 

In [16]:
list(isg.graph.neighbors(11))

[12, 8]

In [18]:
list(isg.graph.to_undirected().neighbors(8))

[7, 23, 9, 11]